In [35]:
import sys
import os
sys.path.append(r"..")

from utils import *
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

path_build = "../../data/tencent2020/build/"
path_save = "../../data/tencent2020/save/"
pickle_path = "../../data/tencent2020/pickle"
sub_path = "../../data/tencent2020/sub"

train_preliminary_p = path_build + "train_preliminary/"

if not os.path.exists(pickle_path):
    os.mkdir(pickle_path) 
if not os.path.exists(sub_path):
    os.mkdir(sub_path)

In [13]:
user_creative_ids = pd.read_pickle(f"{pickle_path}/user_creative_ids.pickle")
user = pd.read_csv(train_preliminary_p + "user.csv", encoding='utf-8')
user_creative_ids
user_creative_ids.head()

,user_id,creative_id
0,1,"[821396, 209778, 877468, 1683713, 122032, 7169..."
1,2,"[63441, 155822, 39714, 609050, 13069, 441462, ..."
2,3,"[661347, 808612, 710859, 825434, 593522, 72694..."
3,4,"[39588, 589886, 574787, 1892854, 1962706, 2264..."
4,5,"[296145, 350759, 24333, 43235, 852327, 1054434..."


In [6]:
cv = CountVectorizer(analyzer='word',token_pattern=u"(?u)\\b\\w+\\b",min_df=1,ngram_range=(1,1))
tf_matrix = cv.fit_transform(user_creative_ids['creative_id'].map(lambda x:' '.join([str(i) for i in x])).values)
print(tf_matrix.shape)


(1900000, 3412772)


AttributeError: head not found

In [7]:
print(type(tf_matrix))

<class 'scipy.sparse.csr.csr_matrix'>


In [10]:
data_tfidf = TfidfTransformer(use_idf=True).fit_transform(tf_matrix)
#clf = MultinomialNB().fit(train_tfidf, user_train['gender'])

In [24]:
gender_clf = MultinomialNB().fit(data_tfidf[0:900000], user['gender'])
age_clf = MultinomialNB().fit(data_tfidf[0:900000], user['age'])

In [28]:
gender_p = gender_clf.predict(data_tfidf[900000:])
age_p = age_clf.predict(data_tfidf[900000:])

In [37]:
sub = pd.DataFrame()
sub['user_id'] = range(3000001,4000001)
sub['predicted_age'] = age_p
sub['predicted_gender'] = gender_p
sub.to_csv(f"{sub_path}/submission.csv", index=False, encoding='utf-8')
print(sub.shape)
sub.head()

(1000000, 3)


,user_id,predicted_age,predicted_gender
0,3000001,3,1
1,3000002,3,2
2,3000003,3,1
3,3000004,3,1
4,3000005,3,1
